In [2]:
!pip install -qU ../input/for-pydicom/python_gdcm-3.0.14-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl ../input/for-pydicom/pylibjpeg-1.4.0-py3-none-any.whl --find-links frozen_packages --no-index

In [3]:
!pip install -q kaggle_vol3d_classify -f ../input/cervical-spine-fracture-detection-npz-3d-volumes/frozen_packages --no-index
# !pip install -qU "pytorch-lightning>1.5.0" --no-index
#!pip uninstall -y torchtext
#!pip list | grep -e lightning -e kaggle -e monai

In [4]:
# ---- efficientNet3D offline  ---
import torch.nn as nn
import sys
sys.path.append('../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D')
from efficientnet_pytorch_3d import EfficientNet3D

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import cv2
import os
from os import listdir
import re
import gc
import random
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.auto import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import warnings

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)
import zipfile
from scipy import ndimage
from sklearn.model_selection import train_test_split, GroupKFold
from joblib import Parallel, delayed
from PIL import Image
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from kaggle_volclassif.utils import interpolate_volume
from skimage import exposure

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
import kornia
import kornia.augmentation as augmentation
warnings.filterwarnings('ignore')

In [6]:
# Set random seeds
def set_seed(seed=0):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
set_seed()

BATCH_SIZE = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [7]:
# Load metadata
train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")
train_bbox = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train_bounding_boxes.csv")
test_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/test.csv")
ss = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv")

debug = False
if len(ss)==3:
    debug = True
    
    # Fix mismatch with test_images folder
    test_df = pd.DataFrame(columns = ['row_id','StudyInstanceUID','prediction_type'])
    for i in ['1.2.826.0.1.3680043.22327','1.2.826.0.1.3680043.25399','1.2.826.0.1.3680043.5876']:
        for j in ['C1','C2','C3','C4','C5','C6','C7','patient_overall']:
            test_df = test_df.append({'row_id':i+'_'+j,'StudyInstanceUID':i,'prediction_type':j},ignore_index=True)
    
    # Sample submission
    ss = pd.DataFrame(test_df['row_id'])
    ss['fractured'] = 0.5
    
    display(test_df.head(3))

In [8]:
# Convert dicom images to 3d tensor
def convert_volume(dir_path, out_dir = "test_volumes", size = (224, 224, 224)):
    ls_imgs = glob(os.path.join(dir_path, "*.dcm"))
    ls_imgs = sorted(ls_imgs, key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))

    imgs = []
    for p_img in ls_imgs:
        dicom = pydicom.dcmread(p_img)
        img = apply_voi_lut(dicom.pixel_array, dicom)
        img = cv2.resize(img, size[:2], interpolation=cv2.INTER_LINEAR)
        imgs.append(img.tolist())
    vol = torch.tensor(imgs, dtype=torch.float32)

    vol = (vol - vol.min()) / float(vol.max() - vol.min())
    vol = interpolate_volume(vol, size).numpy()
    
    # https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_adapt_hist_eq_3d.html
    vol = exposure.equalize_adapthist(vol, kernel_size=np.array([64, 64, 64]), clip_limit=0.01)
    # vol = exposure.equalize_hist(vol)
    vol = np.clip(vol * 255, 0, 255).astype(np.uint8)
    
    path_pt = os.path.join(out_dir, f"{os.path.basename(dir_path)}.pt")
    torch.save(torch.tensor(vol), path_pt)

In [9]:
os.mkdir('/kaggle/working/test_volumes')

# Get paths
ls_dirs = [p for p in glob(os.path.join("../input/rsna-2022-cervical-spine-fracture-detection", "test_images", "*")) if os.path.isdir(p)]
print(f"volumes: {len(ls_dirs)}")

# Convert volumes
_= Parallel(n_jobs=3)(delayed(convert_volume)(p_dir, out_dir='/kaggle/working/test_volumes') for p_dir in tqdm(ls_dirs))

In [10]:
# Dataset for test set only
class RSNADataset(Dataset):
    # Initialise
    def __init__(self, subset='test', df_table=test_df):
        super().__init__()
        
        self.subset = subset
        self.df_table = df_table
        
        # Image paths
        self.volume_dir = '/kaggle/working/test_volumes/'
        
    # Get item in position given by index
    def __getitem__(self, index):
        
        # load 3d volume
        patient = self.df_table.loc[index,'StudyInstanceUID']
        path = os.path.join(self.volume_dir, f'{patient}.pt')
        vol = torch.load(path).to(torch.float32)
        
        return (vol.unsqueeze(0), patient)

    # Length of dataset
    def __len__(self):
        return len(self.df_table)

In [11]:
# Test dataset
test_table = pd.DataFrame(pd.unique(test_df['StudyInstanceUID']),columns=['StudyInstanceUID'])
test_dataset = RSNADataset(subset='test', df_table = test_table)

In [12]:
# Dataloader
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [13]:
model = EfficientNet3D.from_name("efficientnet-b2", override_params={'num_classes': 8}, in_channels=1)
model.load_state_dict(torch.load('../input/effnet-all-trained/EffV2_model.pt'))
model.eval()
model.to(device)

In [14]:
test_df['fractured']=0.5
with torch.no_grad():
    # Loop over batches
    for i, (imgs, patient) in enumerate(test_loader):
        print(f'Iteration {i+1}/{len(test_loader)}')
        # Send to device
        imgs = imgs.to(device)
        
        # Make predictions
        preds = model(imgs)
        
        # Apply sigmoid
        sig = nn.Sigmoid()
        preds = sig(preds)
        preds = preds.to('cpu')
        
        # Save preds
        test_df.loc[test_df['StudyInstanceUID']==patient[0],'fractured'] = preds.numpy().squeeze()
        
print('Inference complete!')

In [15]:
submission = test_df[['row_id','fractured']]
submission.to_csv('submission.csv', index=False)
submission.head(3)